### model 임포트




In [ ]:
!pip install langchain_google_genai langchain_groq

In [ ]:
import os

os.environ["GROQ_API_KEY"]=""
os.environ["TAVILY_API_KEY"]=""
os.environ["GOOGLE_API_KEY"]=""

In [46]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)
llm_groq = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2
)



In [ ]:
result = llm.invoke("langgraph에 관한 발라드 음악을 작사해주세요")
print(result.content)

In [ ]:
result = llm_groq.invoke("langgraph에 관한 발라드 음악을 작사해주세요")
print(result.content)

### Retriever 만들기

In [48]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [ ]:
!pip install langchain_community

In [ ]:
!pip install chromadb

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

#3개의 사이트에서 가져온 문서를 chunck 단위로 나눈것을 list

docs = [WebBaseLoader(url).load() for url in urls]
docs_list =[item for sublist in docs for item in sublist]
# docs_list

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=10
)

doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embeddings,
    collection_name="langgraph",
    persist_directory="./chroma_db"
)

retriever = vectorstore.as_retriever()


### LangChain RAG

In [52]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

#promt
promt = hub.pull("rlm/rag-prompt")

#Post preprocessing
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

#rag chain
rag_chain = promt | llm | StrOutputParser()
rag_chain

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x786ae820bd50>, default_metadata=(), model_kwargs={})
| StrOutputParser

### RAG TEST

In [53]:
#run
question = "tell me about agent memory"
generation = rag_chain.invoke({"context":docs, "question":question})
print(generation)

Agent memory in LLMs involves short-term and long-term storage. Short-term memory uses in-context learning, while long-term memory retains information over extended periods using external vector stores for fast retrieval. This external memory helps overcome the limitations of finite attention spans.


### grade document class

In [54]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
  """
  Binary score for relevance check on retrieved documents.
  """
  binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")


In [55]:
structured_llm_grader =llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

ko_system = """
    검색된 문서와 사용자 질문의 관련성을 평가하는 채점자입니다.\n
    문서에 질문과 관련된 키워드 또는 의미론적 의미가 포함된 경우 관련성이 있는 것으로 평가합니다.\n
    문서가 질문과 관련이 있는지 여부를 나타내기 위해 이진 점수 '예' 또는 '아니오'를 제공합니다.
"""

grade_promt =ChatPromptTemplate.from_messages(
    {
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    }
)

retriever_grader = grade_promt | structured_llm_grader

In [56]:
question = "tell me about agent memory"
docs = retriever.get_relevant_documents(question)
docs_txt = docs[0].page_content
print(retriever_grader.invoke({"question":question , "document":docs_txt}))

<ipython-input-56-3ed1023c1372>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


binary_score='yes'


In [57]:
question = "tell me about agent seoul"
docs = retriever.get_relevant_documents(question)
docs_txt = docs[0].page_content
print(retriever_grader.invoke({"question":question , "document":docs_txt}))

binary_score='no'


### question Rewriter

In [58]:
### Question Re-writer
# Prompt
from langchain_core.prompts import ChatPromptTemplate

system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""


re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [59]:
question_rewriter.invoke({"question":question})

'Here are a few options for improved questions, depending on what aspect of "Agent Seoul" you\'re most interested in:\n\n*   **"Who is Agent Seoul?"** (This is good if you want to know the identity of the person/character/entity known as Agent Seoul)\n*   **"Agent Seoul: Background and Information"** (This is a more general query, suitable if you want a broad overview)\n*   **"Agent Seoul: What is the real name?"** (This is a good question if you are trying to find out the real name of the person who is known as "Agent Seoul"\n*   **"Agent Seoul: Career details"** (This is a good question if you are trying to find out the career details of the person who is known as "Agent Seoul"\n*   **"Agent Seoul: Is Agent Seoul a real person?"** (This is a good question if you are trying to find out if Agent Seoul is a real person)'

In [60]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [ ]:
web_search_tool({"query":"tell me about Taj Mahal"})[1]

### langgrap 실습 및 구현

### 웹페이지 로드 및 벡터 DB 구축함수

In [62]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)
llm_groq = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2
)


In [63]:
!pip  install faiss-cpu

In [98]:
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]


def setup_vectordb(urs:List):
  docs = [WebBaseLoader(url).load() for url in urs]
  docs_list =[item for sublist in docs for item in sublist]
  # docs_list

  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      chunk_size=250, chunk_overlap=10
  )

  doc_splits = text_splitter.split_documents(docs_list)
  vector_store = FAISS.from_documents(doc_splits , embedding=embeddings)
  vector_store.save_local("/content/drive/MyDrive/DB/faiss_index")
  vectorstore = Chroma.from_documents(
      documents=doc_splits,
      embedding=embeddings,
      collection_name="langgraph",
      persist_directory="/content/drive/MyDrive/DB/chroma_db"
  )

  retriever = vectorstore.as_retriever()
  return retriever


In [89]:
# RAG 체인 설정
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 문서 관련성 평가를 위한 구조체
from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# 문서 평가 설정
def setup_grader(llm):
    structured_llm_grader = llm.with_structured_output(GradeDocuments)

    system = """You are a grader assessing relevance of a retrieved document to a user question.
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

    grade_prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ])

    return grade_prompt | structured_llm_grader

In [90]:
# 질문 재작성 설정
def setup_question_rewriter(llm):
    system = """You are a question re-writer that converts an input question to a better version that is optimized
    for web search. Look at the input and try to reason about the underlying semantic intent / meaning.

    Your task is to improve the question to maximize the chance of finding relevant information. Focus on:
    1. Clarifying ambiguous terms
    2. Adding specific keywords related to the topic
    3. Phrasing it in a way that would match informational content

    Keep the question concise and focused on the main topic.
    """

    re_write_prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question that will get the best search results.",
        ),
    ])

    return re_write_prompt | llm | StrOutputParser()

### langgraph 아키텍쳐 구현

In [26]:
# node 정의
def retrieve(state:GraphState):
  print("---RETRIEVE NODE")
  return GraphState(question=question, documents=documents)

def grade_documents(state:GraphState):
  print("---CHECKING DOCUMENT RELEVANT IS TO QUESTION OR NOT---")
  web_search ="yes"
  filtered_docs= []
  return GraphState(question=question,web_search=web_search,documents=filtered_docs,)

def generate(state:GraphState):
  print("---GENERATE---")
  return GraphState(question="question",orginal_question="orginal_question", documents=["Hello"], generation=["AAA"])

def transform_query(state:GraphState):
  print("---TRANSFORM QUERY---")
  return GraphState(question="better question",orginal_question="orginal_question", documents=["web Hello"])

def web_search(state:GraphState):
  print("---WEB SEARCH---")
  return GraphState(question="better question",orginal_question="orginal_question", documents=["web Hello"])


# node 추가
from langgraph.graph import END, StateGraph ,START

workflow = StateGraph(GraphState)
workflow.add_node("retrieve_node", retrieve)
workflow.add_node("grade_documents_node", grade_documents)
workflow.add_node("generate_node", generate)
workflow.add_node("transform_query_node", transform_query)
workflow.add_node("web_search_node", web_search)

#node를 edge로 연결
workflow.add_edge(START , "retrieve_node")
workflow.add_edge("retrieve_node" , "grade_documents_node")
workflow.add_conditional_edges(
    "grade_documents_node", # 시작 노드 이름
    decide_to_generate, # 조건 함수
    {
        "웹검색":"transform_query_node",
        "결과조합":"generate_node" # 결과 키:노드 매핑
    }
)
workflow.add_edge("transform_query_node" , "web_search_node")
workflow.add_edge("web_search_node" , "generate_node")

app = workflow.compile()

#### state 정의

In [91]:
from typing import List , Dict , Any ,TypedDict ,Annotated
from langchain.schema import Document

#전달해주는 state 상태정의
class State(TypedDict):
  question:str
  orginal_question:str
  documents:List[Document]
  web_search:str
  generation:str
  web_results:List[Dict[str,Any]]
  relevance_score:str

class GraphState(State):
  question:Annotated[str , "user qeustion" ]
  documents:Annotated[List[Document] , []]
  orginal_question:Annotated[str , "original question" ]
  web_search:Annotated[str , "web search" ]
  generation:Annotated[str , "generation" ]
  web_results:Annotated[List[Dict[str,Any]] , [{}]]
  relevance_score:Annotated[str , "relevance_score" ]

GraphState(documents=["fewpfjwepfjwpejf" , "fewfewfwefwef"])

{'documents': ['fewpfjwepfjwpejf', 'fewfewfwefwef']}

### Node 정의


In [66]:
#retrieve 정의함수
def retrieve(state:GraphState):
  print("---Retrieve---")
  question = state["question"]

  #검색 실행
  documents = retriever.get_relevant_documents(question)
  # return State(documents=[f"jfwpjfpewpfwejpfjpwef{question}"])
  return GraphState(question=question, documents=documents)

In [67]:
documents = retrieve(State(question="tell me about SEOUL"))
# documents

---Retrieve---


In [68]:
#grade 노드
def grade_documents(state:GraphState):
  """
  check document relevance to question
  """
  print("---CHECKING DOCUMENT RELEVANT IS TO QUESTION OR NOT---")
  question = state["question"]
  documents = state["documents"]

  #문서가 없으면 바로 웹 검색하도록
  if not documents:
    return GraphState(
        question=question,
        orginal_question=state.get("original_questrion", question),
        documents=[],
        web_search="yes",
        web_results=[{}],
        relevance_score="no")

  # 문서 평가
  filtered_docs = []
  web_search ="no"
  relevant_count = 0
  for document in documents:
    score= retriever_grader.invoke(
        {
            "question":question,
            "document":document.page_content
        }
    )
    grade = score.binary_score
    if grade == "yes":
      print("---GRADE : DOCUMENT RELEVANT---")
      filtered_docs.append(document)
      relevant_count +=1
    else:
      print("---GRADE : DOCUMENT NOT RELEVANT---")

  #관련성 너무 적으면 웹서치 할 수 있도록 함
  if relevant_count < 2:
    web_search = "yes"
    print(f"---ONLY {relevant_count} RELEVANT DOCUMENTS, WEB SEARCH NEEDED---")
  # return GraphState(
  #     question=question,
  #     orginal_question=state.get("original_questrion", question),
  #     documents=filtered_docs,
  #     web_search=web_search,
  #     web_results=[{}],
  #     relevance_score="yes" if filtered_docs else "no"
  # )

  return GraphState(question=question,web_search=web_search,documents=filtered_docs,)




In [92]:
from langchain_core.prompts import ChatPromptTemplate
def transform_query(state:GraphState):
  """질문 재작성"""
  print("---TRANSFORM QUERY---")
  system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""


  re_write_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system),
          (
              "human",
              "Here is the initial question: \n\n {question} \n Formulate an improved question.",
          ),
      ]
  )

  question_rewriter = re_write_prompt | llm | StrOutputParser()
  #처음 질문 저장
  original_question = state.get("original_question", state["question"])
  question = state["question"]
  documents = state["documents"]
  #질문 재작성
  better_question = question_rewriter.invoke({"question":question})
  return GraphState(
      question=better_question ,
      orginal_question=original_question,
      documents=documents
      )

In [93]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=5)
def web_search(state:GraphState):
  print("---WEB SEARCH---")
  question = state["question"]
  original_question = state.get("original_question", question)
  print(f"---SEARCHING WEB FOR :{question}---")
  search_results = web_search_tool.invoke({"query":question})

  #검색 결과 처리
  if not search_results:
    print("---NO WEB RESULTS FOUND---")
    return GraphState(
        question=question,
        orginal_question=original_question,
        documents=[],
        web_search="Complete",
        web_results=[],
        relevance_score="no"
    )
  web_docs = []
  for result in search_results:
    content = result.get("content" , "")
    title = result.get("title","")
    if content:
      metadata = {"source" : result.get("url","") , "title":title}
      web_docs.append(Document(page_content=content , metadata=metadata))
  print(f"---FOUND {len(web_docs)} WEB RESULTS---")
  return GraphState(question=question ,orginal_question=original_question, documents=state["documents"], web_search="Complete",web_results=search_results)

In [102]:
from langchain import hub
def generate(state):
    """답변 생성"""
    print("---GENERATE---")

    question = state["original_question"] if "original_question" in state else state["question"]
    documents = state["documents"]

    # RAG 프롬프트 준비
    prompt = hub.pull("rlm/rag-prompt")

    # 검색 결과가 없는 경우 처리
    if not documents:
        return {
            "question": question,
            "original_question": state.get("original_question", question),
            "documents": [],
            "web_search": state.get("web_search", "No"),
            "web_results": state.get("web_results", []),
            "generation": "I couldn't find any relevant information to answer your question."
        }

    # RAG 체인으로 답변 생성
    rag_chain = prompt | llm | StrOutputParser()
    generation = rag_chain.invoke({"context": documents, "question": question})

    # 웹 검색 결과 사용 여부 표시
    if state.get("web_search") == "Completed" and state.get("web_results"):
        sources = []
        for idx, result in enumerate(state.get("web_results", [])[:3], 1):
            if "url" in result:
                sources.append(f"{idx}. {result.get('title', 'Source ' + str(idx))}: {result['url']}")

        if sources:
            generation += "\n\nSources:\n" + "\n".join(sources)

    return {
        "question": question,
        "original_question": state.get("original_question", question),
        "documents": documents,
        "web_search": state.get("web_search", "No"),
        "web_results": state.get("web_results", []),
        "generation": generation
    }

In [95]:
def decide_to_generate(state:GraphState):
  print("---ACCESS GRADES DOCUMENS---")

  web_search = state.get("web_search", "no")

  if web_search == "yes":
    print("---DOCUMENT NOT RELEVANT! WEB SEARCH NEEDED---")
    return "웹검색"
  elif web_search == "Complete":
    print("--- DECISION : WEB SEARCH COMPLETE , GENERATE---")
    return "결과조합"
  else:
    print("--- DECSION! GENERATE---")
    return "결과조합"

In [100]:
# 설정 및 실행 함수
def setup_and_run(user_question, example_urls=None):
    global retriever, retrieval_grader, question_rewriter, web_search_tool

    # 벡터 DB 설정
    if example_urls:
        retriever = setup_vectordb(example_urls)
    else:
        # 기본 URL
        default_urls = [
            "https://lilianweng.github.io/posts/2023-06-23-agent/",
            "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
            "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
        ]
        retriever = setup_vectordb(default_urls)

    # 그레이더 설정
    retrieval_grader = setup_grader(llm)

    # 질문 재작성기 설정
    question_rewriter = setup_question_rewriter(llm)

    # 웹 검색 도구 설정
    web_search_tool = TavilySearchResults(k=5)

    # 그래프 구성
    workflow = StateGraph(State)

    # 노드 추가
    workflow.add_node("retrieve", retrieve)
    workflow.add_node("grade_documents", grade_documents)
    workflow.add_node("generate", generate)
    workflow.add_node("transform_query", transform_query)
    workflow.add_node("web_search_node", web_search)

    # 엣지 추가
    workflow.add_edge(START, "retrieve")
    workflow.add_edge("retrieve", "grade_documents")
    workflow.add_conditional_edges(
        "grade_documents",
        decide_to_generate,
        {
            "웹검색": "transform_query",
            "결과조합": "generate",
        }
    )
    workflow.add_edge("transform_query", "web_search_node")
    workflow.add_edge("web_search_node", "generate")
    workflow.add_edge("generate", END)

    # 그래프 컴파일
    app = workflow.compile()

    # 실행
    inputs = {"question": user_question}
    result = app.invoke(inputs)

    # 결과 반환
    return result["generation"]

In [104]:
setup_and_run("tell me about SEOUL")

---Retrieve---
---CHECKING DOCUMENT RELEVANT IS TO QUESTION OR NOT---
---GRADE : DOCUMENT NOT RELEVANT---
---GRADE : DOCUMENT NOT RELEVANT---
---GRADE : DOCUMENT NOT RELEVANT---
---GRADE : DOCUMENT NOT RELEVANT---
---ONLY 0 RELEVANT DOCUMENTS, WEB SEARCH NEEDED---
---ACCESS GRADES DOCUMENS---
---DOCUMENT NOT RELEVANT! WEB SEARCH NEEDED---
---TRANSFORM QUERY---
---WEB SEARCH---
---SEARCHING WEB FOR :Here are a few options for improved search queries, depending on the specific intent:

*   **General Overview:** "Seoul city guide" or "Seoul travel information"
*   **History:** "History of Seoul South Korea"
*   **Culture:** "Culture in Seoul" or "Traditions of Seoul"
*   **Tourism:** "Top attractions in Seoul" or "Things to do in Seoul"
*   **Current Events:** "News from Seoul South Korea"

I'd recommend using "Seoul travel guide" as this is a great starting point.---


AttributeError: 'str' object has no attribute 'get'

In [72]:
transform_query(grade_state)

---TRANSFORM QUERY---


{'question': 'Here are a few options for rewritten questions, depending on the specific intent behind the original question:\n\n**Option 1 (General Information):**\n\n*   **Search Query:** "Seoul South Korea information"\n\n    *Reasoning:* This is a broad search aimed at getting general facts and details about the city.\n\n**Option 2 (History):**\n\n*   **Search Query:** "History of Seoul"\n\n    *Reasoning:*  If the user is interested in the historical background of Seoul.\n\n**Option 3 (Tourism):**\n\n*   **Search Query:** "Tourist attractions in Seoul"\n\n    *Reasoning:* If the user is planning a trip and wants to know what to see.\n\n**Option 4 (Culture):**\n\n*   **Search Query:** "Culture and traditions in Seoul"\n\n    *Reasoning:* If the user is interested in the cultural aspects of the city.\n\n**Option 5 (Modern Seoul):**\n\n*   **Search Query:** "Modern Seoul architecture and lifestyle"\n\n    *Reasoning:* If the user is interested in contemporary aspects of the city.\n\n*

In [85]:
prompt = hub.pull("rlm/rag-prompt")
prompt_1 = """
      You are an assistant for question-answering tasks.
      Use the following pieces of retrieved context to answer the question.
      If you don't know the answer,
      just say that i am a stupid.
      Use five sentences maximum and keep the answer concise.\n

      Question: {question} \n


      Context: {context} \nAnswer:

    """

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
from IPython.display import Image , display
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
!pip install langgraph

In [70]:
### Question Re-writer
# Prompt
from langchain_core.prompts import ChatPromptTemplate

system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""


re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [76]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=5)
state = web_search_tool.invoke({"query":"서울에 대해서"})[0]

In [79]:
transform_query({"question":"tell me about Busan","original_question":"tell me about SEOUL","documents":["Hello"]})

---TRANSFORM QUERY---


{'question': 'Here are a few options for rephrasing the question "tell me about Busan" for optimal web search, depending on the specific information desired:\n\n**Option 1 (General Overview):**\n\n*   **Best:** "Busan South Korea travel guide"\n\n**Reasoning:** This targets travel-related information, which is a common reason for people to search for information about a city.\n\n**Option 2 (Broader Information):**\n\n*   **Good:** "Busan South Korea information"\n\n**Reasoning:** This is a broad but direct query that should return a wide range of results about Busan, including its history, culture, and economy.\n\n**Option 3 (If you suspect the user is unfamiliar with Busan):**\n\n*   **Good:** "Where is Busan? What is Busan known for?"\n\n**Reasoning:** This explicitly asks for location and key characteristics.\n\nThe best option depends on what the user is *most likely* trying to find out. Without more context, I would recommend "Busan South Korea travel guide" as the most likely to 

In [69]:
grade_state = grade_documents(documents)

---CHECKING DOCUMENT RELEVANT IS TO QUESTION OR NOT---
---GRADE : DOCUMENT NOT RELEVANT---
---GRADE : DOCUMENT NOT RELEVANT---
---GRADE : DOCUMENT RELEVANT---
---GRADE : DOCUMENT NOT RELEVANT---
---ONLY 1 RELEVANT DOCUMENTS, WEB SEARCH NEEDED---


In [ ]:
web_search({})